In [12]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [13]:
from transformers import pipeline

In [14]:
model_checkpoint = "brad1141/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)

In [15]:
def calc_word_indices(full_text, discourse_start, discourse_end):
    start_index = len(full_text[:discourse_start].split())
    token_len = len(full_text[discourse_start:discourse_end].split())
    output = list(range(start_index, start_index + token_len))
    if output[-1] >= len(full_text.split()):
        output = list(range(start_index, start_index + token_len-1))
    return output

In [18]:
def predict_and_format(fileName):
    data = ""
    with open('test/' + fileName, 'r') as file:
        data = file.read().replace('\n', '')
    
    predicts = token_classifier(data)

    for p in predicts:
        word_Indices_Array = calc_word_indices(data, p["start"], p["end"])
        word_Indices_String = " ".join(str(x) for x in word_Indices_Array)
        word_class = p["entity_group"]
        word_id = fileName[:-4]
        if len(word_Indices_Array) >= 3:
            sub_df.loc[len(sub_df.index)] = [word_id, word_class, word_Indices_String]

In [19]:
dict = {'id':[],
        'class': [],
        'predictionstring':[]}
sub_df = pd.DataFrame(dict)

fileNames = []
# Replace with your folder
for file in os.walk('/kaggle/input/feedback-prize-2021/test/'):
    fileNames = file[2]
    break

print(fileNames)
for file in fileNames:
    predict_and_format(file)

['0FB0700DAF44.txt', '18409261F5C2.txt', 'D46BCB48440A.txt', 'D72CB1C11673.txt', 'DF920E0A7337.txt']


In [20]:
sub_df.to_csv('submission2.csv',index=False)